<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 5

Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [1]:
using LinearAlgebra
using Plots, Printf

function print_matrix(M, decimals::Int=4)
  for i in 1:size(M, 1)
    for j in 1:size(M, 2)
      @printf("%*.*f ", 10, decimals, M[i, j])
    end
    println()
  end
end

print_matrix (generic function with 2 methods)

Esse trecho de código é usado apenas para garantir a repetibilidade dos números aleatórios gerados.

In [2]:
using Random
Random.seed!(123)

TaskLocalRNG()

### Questão 1

Primeiro iremos gerar os 100 pontos aleatórios $x_i$ e construir $X$.

In [3]:
n = 100
X = rand(2, n)

2×100 Matrix{Float64}:
 0.9063    0.745673  0.253849  0.427328  …  0.31068   0.93875   0.0904866
 0.443494  0.512083  0.334152  0.867547     0.786312  0.918775  0.688171

Sabemos que
$$
\|x_i - x_j\|^2 = \|x_i\|^2 + \|x_j\|^2 - 2 x_i^\top x_j.
$$
Logo, pode-se construir a matriz $D$ da seguinte forma:

In [8]:
XT = transpose(X)
norms_squared = sum(XT.^2, dims=2)
D = norms_squared .+ norms_squared' .- 2 .* (XT * X)

100×100 Matrix{Float64}:
 0.0         0.0305053   0.437647     …   0.472287     0.226945    0.725418
 0.0305053   0.0         0.273551         0.26442      0.202677    0.460277
 0.437647    0.273551   -5.55112e-17      0.207679     0.810874    0.152017
 0.409235    0.227699    0.314606         0.0202057    0.264177    0.145638
 0.752772    0.567625    0.0675611        0.481706     1.33458     0.316912
 0.140048    0.143882    0.231205     …   0.567753     0.672656    0.666353
 0.77287     0.535286    0.0494429        0.267501     1.14491     0.1174
 0.266772    0.234221    0.848607         0.41394      0.00172399  0.778925
 0.0850472   0.108738    0.293332         0.578067     0.567661    0.721953
 0.457266    0.332279    0.0357187        0.404883     1.00259     0.332118
 0.00801747  0.0465803   0.40963      …   0.525866     0.320036    0.755981
 0.847497    0.560927    0.383323         0.0559683    0.651672    0.0656967
 0.760282    0.54512     0.0465427        0.356619     1.2304   

### Questão 2

Primeiro iremos construir a matriz $J = I - \frac{1}{n} e e^T$

In [11]:
I_n = Matrix{Float64}(I, n, n)
e = ones(n)
eeT = e * e'
J = I_n - (1/n) * eeT;

Agora, calculemos $G= -\frac{1}{2}JDJ$

In [18]:
G = -0.5 * J * D * J;

A matriz $G$ contém erros de ponto flutuante. Portanto, estarei forçando que a matriz seja real e simétrica para eliminar pequenas partes imaginárias que prejudicam a decomposição espectral.

In [22]:
G = real.(G)
G = (G + G') / 2;

Para obter $Y=\sqrt\Lambda_rQ_r^T$, precisamos encontrar a decomposição espectral $G=Q \Lambda Q^T$.

In [28]:
function decomposicao_espectral(G::AbstractMatrix; tol=1e-10)
    # Decomposição espectral
    eig = eigen(G)
    lamb = eig.values
    Q = eig.vectors

    # Ordena autovalores em ordem decrescente
    idxs = sortperm(lamb, rev=true)
    lamb_sorted = lamb[idxs]
    Q_sorted = Q[:, idxs]

    # Seleciona autovalores positivos
    inds_pos = findall(lamb_sorted .> tol)
    Lambda_r = Diagonal(lamb_sorted[inds_pos])
    Q_r = Q_sorted[:, inds_pos]

    return Q_r, Lambda_r
end

decomposicao_espectral (generic function with 1 method)

In [32]:
Q_r, Lambda_r = decomposicao_espectral(G)
Y = sqrt.(Lambda_r) * Q_r'

2×100 Matrix{Float64}:
 -0.37341   -0.23282   0.286717  …   0.1276    -0.514105   0.364444
  0.146956   0.043323  0.103594     -0.323444  -0.308182  -0.278473

### Questão 3

In [37]:
Xc = X*J
Yc = Y*J

2×100 Matrix{Float64}:
 -0.37341   -0.23282   0.286717  …   0.1276    -0.514105   0.364444
  0.146956   0.043323  0.103594     -0.323444  -0.308182  -0.278473

Veja que esse é um problema de Procrustes Ortogonal cuja solução é dada ao calcular a SVD de $Y_c X_c^T = W \Sigma V^T$ e a matriz ótima $U=WV^T$.

In [40]:
F = svd(Yc * Xc')
U = F.U * F.Vt
residual = norm(U * Yc - Xc, 2)

1.8447334060791967

Observe que, de fato, o a matriz $U$ é ortogonal com excessão de algum resíduo.

In [43]:
U*U'

2×2 Matrix{Float64}:
 1.0          2.77556e-17
 2.77556e-17  1.0

### Questão 4